In [1]:
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

from datasets import load_dataset
from grammar_induction import *
import re
from collections import defaultdict

In [2]:
class temp():
    sample_size = 0
    max_iterations = 10000
    min_delta = 0
    terminal_codelength = 8
    non_terminal_codelength = 1
    parse_sample = True
    allow_repeated_target_nts = True
    seed_exact_match = True
    balance_parens = True
FLAGS = temp()
config = induction_utils.InductionConfig(
    sample_size=FLAGS.sample_size,
    max_iterations=FLAGS.max_iterations,
    min_delta=FLAGS.min_delta,
    terminal_codelength=FLAGS.terminal_codelength,
    non_terminal_codelength=FLAGS.non_terminal_codelength,
    parse_sample=FLAGS.parse_sample,
    allow_repeated_target_nts=FLAGS.allow_repeated_target_nts,
    seed_exact_match=FLAGS.seed_exact_match,
    balance_parens=FLAGS.balance_parens,
)
to_tuple_list = lambda x: [(s, s) for s in x]

In [3]:
examples = load_dataset("scan", "length")
examples = [(inst['commands'], inst['actions']) for inst in examples['train']]

Reusing dataset scan (/media/disk1/ksk5693/huggingface/datasets/scan/length/1.0.0/a4eda004484cb2e56cebf1d9555886256eb76d369550a48b9eb052900ba7148a)


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
from grammar_induction import exact_match_utils
seed_rules = exact_match_utils.get_exact_match_rules(examples)
seed_rules

set()

In [5]:
rule = qcfg_rule.QCFGRule(source=tuple(['walk']), target=tuple(['I_WALK']), arity=0)
seed_rules.add(rule)
rule = qcfg_rule.QCFGRule(source=tuple(['jump']), target=tuple(['I_JUMP']), arity=0)
seed_rules.add(rule)
rule = qcfg_rule.QCFGRule(source=tuple(['look']), target=tuple(['I_LOOK']), arity=0)
seed_rules.add(rule)
rule = qcfg_rule.QCFGRule(source=tuple(['run']), target=tuple(['I_RUN']), arity=0)
seed_rules.add(rule)

In [6]:
seed_rules

{jump ### I_JUMP, look ### I_LOOK, run ### I_RUN, walk ### I_WALK}

In [7]:
induced_rules = induction_utils.induce_rules(examples[:100], config)

Added 100 seed rules for examples.
Seed rules after adding exact match rules for all examples: 100.
Updating rule 1 of 100.
Updating rule 11 of 100.
Updating rule 21 of 100.
Updating rule 31 of 100.
Updating rule 41 of 100.
Updating rule 51 of 100.
Updating rule 61 of 100.
Updating rule 71 of 100.
Updating rule 81 of 100.
Updating rule 91 of 100.
Updates complete.
Iteration 0.
Candidate rules:
turn left after NT_1 ### NT_1 I_TURN_LEFT (278.0)
NT_1 after jump ### I_JUMP NT_1 (246.0)
NT_1 and look ### NT_1 I_LOOK (246.0)
run and NT_1 ### I_RUN NT_1 (246.0)
NT_1 after turn left ### I_TURN_LEFT NT_1 (214.0)
NT_1 after turn right ### I_TURN_RIGHT NT_1 (214.0)
NT_1 and turn left ### NT_1 I_TURN_LEFT (214.0)
NT_1 and turn right ### NT_1 I_TURN_RIGHT (214.0)
turn left and NT_1 ### I_TURN_LEFT NT_1 (214.0)
turn right after NT_1 ### NT_1 I_TURN_RIGHT (214.0)
turn right and NT_1 ### I_TURN_RIGHT NT_1 (214.0)
NT_1 twice ### NT_1 NT_1 (197.0)
NT_1 after look ### I_LOOK NT_1 (190.0)
NT_1 after run #

In [8]:
induced_rules

{NT_1 after NT_2 ### NT_2 NT_1,
 NT_1 and NT_2 ### NT_1 NT_2,
 NT_1 left ### I_TURN_LEFT NT_1,
 NT_1 opposite ### I_TURN_LEFT NT_1,
 NT_1 opposite ### I_TURN_RIGHT NT_1,
 NT_1 right ### I_TURN_RIGHT NT_1,
 NT_1 twice ### NT_1 NT_1,
 jump ### I_JUMP,
 look ### I_LOOK,
 run ### I_RUN,
 turn left ### I_TURN_LEFT,
 turn opposite left ### I_TURN_LEFT I_TURN_LEFT,
 turn opposite right ### I_TURN_RIGHT I_TURN_RIGHT,
 turn right ### I_TURN_RIGHT,
 walk ### I_WALK}